<a href="https://colab.research.google.com/github/Deekshithxyz/BigData/blob/main/IPLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install -q findspark

In [ ]:
import findspark

In [ ]:
findspark.init()

Exception: ignored

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, StringType 

In [ ]:
from pyspark.sql.functions import col

In [ ]:
spark = (SparkSession
.builder
.appName("IPL")
.getOrCreate())

In [ ]:
schema_matches = StructType([StructField("match_id",IntegerType(),True),StructField("date",TimestampType(),True),StructField("player_of_match",StringType(),True),StructField("venue_id",IntegerType(),True),StructField("neutral_venue",IntegerType(),True),StructField("team1",StringType(),True),StructField("team2",StringType(),True),StructField("toss_winner",StringType(),True),StructField("toss_decision",StringType(),True),StructField("winner",StringType(),True),StructField("result",StringType(),True),StructField("result_margin",IntegerType(),True),StructField("eliminator",StringType(),True),StructField("method", StringType(),True),StructField("umpire1", StringType(),True),StructField("umpire2", StringType(),True)])

In [ ]:
matches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").schema(schema_matches).load("ipl_matches.csv")

# New Section

In [ ]:
venues = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("ipl_venue.csv")

In [ ]:
ball_by_ball = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("ipl_ball_by_ball.csv")

In [ ]:
matches.createOrReplaceTempView("MATCH")

In [ ]:
venues.createOrReplaceTempView("VENUE")

In [ ]:
ball_by_ball.createOrReplaceTempView("BALL")

In [ ]:
a = spark.sql("select VENUE.venue_id, VENUE.venue, VENUE.city, tvenue.count from VENUE inner join (select venue_id, count(match_id) as count from MATCH where eliminator='Y' group by venue_id order by count desc limit 3) as tvenue on VENUE.venue_id=tvenue.venue_id order by count desc")

In [ ]:
b = spark.sql("select count(fielder) as catches from BALL where dismissal_kind='caught' group by fielder order by catches desc limit 1")

In [ ]:
c = spark.sql("select bowler, count(*) as wickets from BALL where is_wicket=1 and match_id in (select match_id from MATCH where method='D/L') group by bowler order by wickets desc limit 1")

In [ ]:
d = spark.sql("select batsman, sum(batsman_runs)*100/count(*) as strike_rate  from BALL where overs >= 7 and not extras_type='wides' and not extras_type='noballs' group by batsman order by strike_rate desc limit 1")

In [ ]:
e = spark.sql("select sum(A.extra_runs) as extra_runs, first(C.venue) as venue, first(C.city) as city from BALL as A inner join MATCH as B on A.match_id=B.match_id inner join VENUE as C on B.venue_id=C.venue_id group by C.venue_id order by extra_runs desc limit 1")

In [ ]:
f = spark.sql("select * from (select player_of_match, count(*) as count from MATCH where neutral_venue=1 group by player_of_match) where count=(select max(count) from (select count(*) as count from MATCH where neutral_venue=1 group by player_of_match))")

In [ ]:
player_dismissed=ball_by_ball.groupby('player_dismissed').count()

In [ ]:
runs=ball_by_ball.groupBy('batsman').sum('batsman_runs')

In [ ]:
joined_data=player_dismissed.join(runs, player_dismissed.player_dismissed == runs.batsman, 'inner')

In [ ]:
AVG=joined_data.withColumn("Batsmen Avg", col("sum(batsman_runs)")/col("count")).sort('Batsmen Avg',ascending=False)

In [ ]:
g = AVG.select('batsman','Batsmen Avg')

In [ ]:
h = spark.sql("select umpire, count(*) as count from ((select umpire1 as umpire from MATCH) union all (select umpire2 from MATCH)) as matches group by umpire order by count desc")

In [ ]:
i = spark.sql("select B.match_id,first(C.venue),first(C.city),sum(A.batsman_runs) from BALL as A inner join MATCH as B on A.match_id=B.match_id inner join VENUE as C on B.venue_id =C.venue_id where A.batsman = 'V Kohli' group by B.match_id order by sum(A.batsman_runs) desc limit 1")

In [ ]:
j = spark.sql("select sum(count) from (select winner, count(winner) as count from MATCH where toss_winner=winner group by winner)")

In [ ]:
import sqlite3
from sqlite3 import Error

In [ ]:
class Database:

   def __init__(self, m):
    self.path = m
 
   def create_connection(self):
    connection = None
    try:
        connection = sqlite3.connect(self.path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

   def execute_query(self, connect, query):
    cursor = connect.cursor()
    try:
        cursor.execute(query)
        connect.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")